In [1]:
import bimpm

C:\Users\mariya.bakhanova\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%run bimpm.py
%run text_processing_rv.py

In [3]:
# URL of the UDPipe model
udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
udpipe_filename = udpipe_model_url.split('/')[-1]

if not os.path.isfile(udpipe_filename):
    print('UDPipe model not found. Downloading...', file=sys.stderr)
    wget.download(udpipe_model_url)

print('\nLoading the model UDP...', file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')


Loading the model UDP...


In [4]:
x = My_model()
x.sen_len = 13
x.load_w2v()

x.sentence_one,x.sentence_two,x.y_true = load_dataset(x.dataset_path_train,x.word2idx,model, process_pipeline)
x.sentence_one_test,x.sentence_two_test,x.y_true_test = load_dataset_test(x.dataset_path_test,x.word2idx,model,process_pipeline)
# exit(0)
dropout_rate = 0.1
num_epoch = 2
batch_size = 314

Creating word2vec embeddings:
Saving word2idx to: cache\word2idx.pkl
Done!
Saving word2vec embeddings to: cache\bimpm.pkl
Done!
8214 300
Shape of word2vec embeddings: (8214, 300)
Max_train: 11 13
(1910, 13) (1910, 13) (1910,)
Max_test: 11 11
(942, 13) (942, 13) (942,)


In [5]:
x.sentence_one_test.shape

(942, 13)

In [6]:
X_init = tf.placeholder(tf.float32, shape=(x.vocab_size, x.embed_dim)) # (vocab_size, 300)

sentence_one = tf.placeholder("int32",[batch_size, x.sen_len],name="sentence_one")
sentence_two = tf.placeholder("int32",[batch_size, x.sen_len],name="sentence_two")

Y = tf.placeholder("int32",[batch_size, 2],name="true_labels")

embedding_weights = tf.get_variable( # variable
    name = 'embedding_weights',
    initializer = X_init,
    trainable = False)

In [7]:
with tf.name_scope("Word_embeddings"):
    
    # nn - module for constructing rnn cells
    # embedding_lookup - looks up ids in a list of embedding tensors.
    embedded_sentence_one = tf.nn.embedding_lookup(embedding_weights,sentence_one)
    embedded_sentence_two = tf.nn.embedding_lookup(embedding_weights,sentence_two)

In [8]:
with tf.variable_scope("Context_representation_layer"):

    context_rnn_hidden_size = 100

    sentence_enc_fw = tf.nn.rnn_cell.LSTMCell(context_rnn_hidden_size, state_is_tuple=True)
    sentence_enc_bw = tf.nn.rnn_cell.LSTMCell(context_rnn_hidden_size, state_is_tuple=True)

    # Sentence 1
    # Shape (batch_size, sequence_length, rnn_hidden_size)

    outputs_1, states_1  = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=sentence_enc_fw,
        cell_bw=sentence_enc_bw,
        dtype=tf.float32,
        # sequence_length=length(sentence_one),
        inputs=embedded_sentence_one)

    output_fw_1, output_bw_1 = outputs_1
    states_fw_1, states_bw_1 = states_1

    output_fw_1 = tf.layers.dropout(
        output_fw_1,
        rate=dropout_rate,
        training=False,
        name="sentence_1_fw_dropout")
    output_bw_1 = tf.layers.dropout(
        output_bw_1,
        rate=dropout_rate,
        training=False,
        name="sentence_1_bw_dropout")

    tf.get_variable_scope().reuse_variables()

    #Sentence 2

    # Shape (batch_size, sequence_length, rnn_hidden_size)

    outputs_2, states_2  = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=sentence_enc_fw,
        cell_bw=sentence_enc_bw,
        dtype=tf.float32,
        # sequence_length=length(sentence_two),
        inputs=embedded_sentence_two)

    output_fw_2, output_bw_2 = outputs_2
    states_fw_2, states_bw_2 = states_2

    output_fw_2 = tf.layers.dropout(
        output_fw_2,
        rate=dropout_rate,
        training=False,
        name="sentence_2_fw_dropout")
    output_bw_2 = tf.layers.dropout(
        output_bw_2,
        rate=dropout_rate,
        training=False,
        name="sentence_2_bw_dropout")

    tf.get_variable_scope().reuse_variables()

with tf.variable_scope("Matching_layer"):

    # Shape (batch_size, sequence_length, 8*perspectives)

    match_1_2, match_2_1 = bidirectional_match(output_fw_1,output_bw_1,output_fw_2,output_bw_2)

    match_1_2 = tf.layers.dropout(
        match_1_2,
        rate=dropout_rate,
        training=False,
        name="match_1_2_dropout")
    match_2_1 = tf.layers.dropout(
        match_2_1,
        rate=dropout_rate,
        training=False,
        name="match_2_1_dropout")

Instructions for updating:
Use `argmax` instead


In [9]:
## Aggregation Layer

with tf.variable_scope("Aggregation_layer"):

    agg_last = []

    # Sentence 1

    with tf.variable_scope("agg_1",reuse=tf.AUTO_REUSE):

        aggregation_enc_fw = tf.nn.rnn_cell.LSTMCell(context_rnn_hidden_size,state_is_tuple=True)
        aggregation_enc_bw = tf.nn.rnn_cell.LSTMCell(context_rnn_hidden_size,state_is_tuple=True)	

        # Shape (batch_size, sequence_length, rnn_hidden_size)


        agg_outputs_1, agg_states_1  = tf.nn.bidirectional_dynamic_rnn(
            cell_fw=aggregation_enc_fw,
            cell_bw=aggregation_enc_bw,
            dtype=tf.float32,
            # sequence_length=length(sentence_one),
            inputs=match_1_2)

        agg_output_fw_1, agg_output_bw_1 = agg_outputs_1

        agg_output_fw_1 = tf.layers.dropout(
            agg_output_fw_1,
            rate=dropout_rate,
            training=False,
            name="agg_1_fw_dropout")
        agg_output_bw_1 = tf.layers.dropout(
            agg_output_bw_1,
            rate=dropout_rate,
            training=False,
            name="agg_1_bw_dropout")

        # Shape : (batch_size, rnn_hidden_size)

        agg_output_fw_1_last = get_last_output(agg_output_fw_1)
        agg_output_bw_1_last = get_last_output(agg_output_bw_1)

        agg_last.append(agg_output_fw_1_last)
        agg_last.append(agg_output_bw_1_last)

    #tf.get_variable_scope().reuse_variables()

    #Sentence 2

    with tf.variable_scope("agg_2",reuse=tf.AUTO_REUSE):

        aggregation_enc_fw_2 = tf.nn.rnn_cell.LSTMCell(context_rnn_hidden_size,state_is_tuple=True)
        aggregation_enc_bw_2 = tf.nn.rnn_cell.LSTMCell(context_rnn_hidden_size,state_is_tuple=True)

        # Shape (batch_size, sequence_length, rnn_hidden_size)

        agg_outputs_2, agg_states_2  = tf.nn.bidirectional_dynamic_rnn(
            cell_fw=aggregation_enc_fw_2,
            cell_bw=aggregation_enc_bw_2,
            dtype=tf.float32,
            # sequence_length=length(sentence_two),
            inputs=match_2_1)

        agg_output_fw_2, agg_output_bw_2 = agg_outputs_2

        agg_output_fw_2 = tf.layers.dropout(
            agg_output_fw_2,
            rate=dropout_rate,
            training=False,
            name="agg_2_fw_dropout")
        agg_output_bw_2 = tf.layers.dropout(
            agg_output_bw_2,
            rate=dropout_rate,
            training=False,
            name="agg_2_bw_dropout")


        # Shape : (batch_size, rnn_hidden_size)

        agg_output_fw_2_last = get_last_output(agg_output_fw_2)
        agg_output_bw_2_last = get_last_output(agg_output_bw_2)


        agg_last.append(agg_output_fw_2_last)
        agg_last.append(agg_output_bw_2_last)

    # Shape : (batch_size, 4*rnn_hidden_size)

    combined_agg_last = tf.concat(agg_last,1)

In [10]:
##Prediction Layer
with tf.variable_scope("Prediction_layer",reuse=tf.AUTO_REUSE):

    prediction_layer_1 = tf.layers.dense(combined_agg_last,
        combined_agg_last.get_shape().as_list()[1],
        activation=tf.nn.tanh,
        name="pred_1")

    prediction_layer_1 = tf.layers.dropout(
        prediction_layer_1,
        rate=dropout_rate,
        training=False,
        name="pred_dropout")

    prediction_layer_2 = tf.layers.dense(prediction_layer_1,2,name="pred_2")

with tf.variable_scope("Train_loss_and_acc", reuse=tf.AUTO_REUSE):

    y_pred = tf.nn.softmax(prediction_layer_2)
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=prediction_layer_2))

    tf.summary.scalar('loss',loss_op)

    correct_predictions = tf.equal(tf.argmax(y_pred, 1),tf.argmax(Y, 1))
#     correct_positive = (tf.equal(tf.equal(tf.argmax(y_pred, 1),tf.argmax(Y, 1)), tf.cast(tf.ones(batch_size), 'bool')))
    y_pred_argmax = tf.argmax(y_pred, 1)
    y_true_argmax = tf.argmax(Y, 1)
    TP = tf.count_nonzero(y_pred_argmax * y_true_argmax, dtype=tf.float32)
    TN = tf.count_nonzero((y_pred_argmax - 1) * (y_true_argmax - 1), dtype=tf.float32)
    FP = tf.count_nonzero(y_pred_argmax * (y_true_argmax - 1), dtype=tf.float32)
    FN = tf.count_nonzero((y_pred_argmax - 1) * y_true_argmax, dtype=tf.float32)
    
    batch_accuracy = tf.reduce_mean(tf.cast(correct_predictions,"float"))
#     precision_numerator = tf.reduce_mean(tf.cast(correct_positive,"float"))
    
#     batch_precision = tf.metrics.precision(Y, y_pred)
#     batch_recall = tf.metrics.recall(Y, y_pred)
#     batch_f1_score = tf.reduce_mean(tf.divide(tf.multiply(tf.cast(2, "float"), tf.multiply(batch_recall, batch_precision)), tf.reduce_sum(batch_precision, batch_recall)))
    tf.summary.scalar('accuracy',batch_accuracy)
#     tf.summary.scalar('precision_numerator',precision_numerator)
#     tf.summary.scalar('precision',batch_precision)
#     tf.summary.scalar('recall',batch_recall)
#     tf.summary.scalar('f1_score',batch_f1_score)

    optimizer = tf.train.AdamOptimizer(learning_rate=x.learning_rate)
    train_op = optimizer.minimize(loss_op, )

merged = tf.summary.merge_all()
saver = tf.train.Saver()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [11]:
print(tf.argmax(y_pred, 1))
print(y_pred)
print(tf.equal(tf.argmax(y_pred, 1),tf.argmax(Y, 1)))
print(tf.count_nonzero(tf.argmax(y_pred, 1) * tf.argmax(Y, 1)))
print(tf.equal(tf.equal(tf.argmax(y_pred, 1),tf.argmax(Y, 1)), tf.cast(tf.ones(batch_size), 'bool')))


Tensor("ArgMax:0", shape=(314,), dtype=int64)
Tensor("Train_loss_and_acc/Softmax:0", shape=(314, 2), dtype=float32)
Tensor("Equal:0", shape=(314,), dtype=bool)
Tensor("count_nonzero/Sum:0", shape=(), dtype=int64)
Tensor("Equal_2:0", shape=(314,), dtype=bool)


In [12]:
from tqdm import tqdm

In [15]:
with tf.Session() as sess:
    trained_model = os.path.join('bimpm_lstm', 'model_RV.ckpt')

    train_writer = tf.summary.FileWriter("bimpm_lstm/",sess.graph)
    print("Training...")
    sess.run(tf.global_variables_initializer(), feed_dict={X_init:x.weights})

    i = 0.0
    sum_acc = 0.0
    sum_tp = 0.0
    sum_tn = 0.0
    sum_fp = 0.0
    sum_fn = 0.0

    for epoch in tqdm(range(num_epoch)):   #num_epoch

        for step in tqdm(range(int(x.sentence_one.shape[0]/batch_size))): 

            i += 1

            batch_s1,batch_s2,batch_y = x.sentence_one[step*batch_size:(step+1)*batch_size],\
                                        x.sentence_two[step*batch_size:(step+1)*batch_size],\
                                        x.y_true[step*batch_size:(step+1)*batch_size]

            #print batch_s1.shape,batch_s2.shape,batch_y.shape

            batch_y = one_hot(batch_size,batch_y.astype(int))

#             [summary,_,loss,acc, TP, TN, FP, FN] = sess.run([merged,train_op,loss_op,batch_accuracy, TP, TN, FP, FN],
            [summary,_,loss,acc, tp, tn, fp, fn] = sess.run([merged,train_op,loss_op,batch_accuracy, TP, TN, FP, FN],
                            feed_dict={X_init:x.weights,
                            sentence_one: batch_s1,
                            sentence_two: batch_s2,
                            Y:batch_y
                            })

            train_writer.add_summary(summary, i)


            sum_acc += acc
            
            sum_tp += tp
            sum_tn += tn
            sum_fp += fp
            sum_fn += fn
                        
            precision = tp / (tp + fp)
            recall = tp / (tp + fn)
            f1 = 2 * precision * recall / (precision + recall)
            
            sum_precision = sum_tp / (sum_tp + sum_fp)
            sum_recall = sum_tp / (sum_tp + sum_fn)
            sum_f1 = 2 * sum_precision * sum_recall / (sum_precision + sum_recall)
            
            print("CURRENT TP:", tp, " TN:", tn, " FP:", fp, " FN:", fn, " Precision:", precision, " Recall:", recall, " F1-score:", f1)
            print("OVERALL TP:", sum_tp, " TN:", sum_tn, " FP:", sum_fp, " FN:", sum_fn, 
                          " Precision:", sum_precision, " Recall:", sum_recall, " F1-score:", sum_f1)
            
            print("Epoch:" + str(epoch+1) + " Step:" + str(step) + " Loss:" + "{:.4f}".format(loss) + " Batch Acc:" + "{:.4f}".format(acc) + " Mean Batch Acc:" + "{:.4f}".format(sum_acc/i))

#             log("Epoch:" + str(epoch+1) + " Step:" + str(step) + " Loss:" + "{:.4f}".format(loss) + " Batch Acc:" + "{:.4f}".format(acc) + " Mean Batch Acc:" + "{:.4f}".format(sum_acc/i))

    save_path = saver.save(sess, trained_model)
    print("Model saved in path: %s" % save_path)

    print("Testing Model...")
    i = 0
    
    sum_acc = 0.0
    sum_tp = 0.0
    sum_tn = 0.0
    sum_fp = 0.0
    sum_fn = 0.0
    for step in tqdm(range(int(x.sentence_one_test.shape[0]/batch_size))):

        i += 1

        batch_s1,batch_s2,batch_y = x.sentence_one_test[step*batch_size:(step+1)*batch_size],\
                                    x.sentence_two_test[step*batch_size:(step+1)*batch_size],\
                                    x.y_true_test[step*batch_size:(step+1)*batch_size]

        batch_y = one_hot(batch_size,batch_y.astype(int))

#         [acc, TP, TN, FP, FN] = sess.run([batch_accuracy, TP, TN, FP, FN],
        [acc, tp, tn, fp, fn] = sess.run([batch_accuracy, TP, TN, FP, FN],
                        feed_dict={X_init:x.weights,
                        sentence_one: batch_s1,
                        sentence_two: batch_s2,
                        Y:batch_y
                        })
        sum_acc += acc
        
        sum_tp += tp
        sum_tn += tn
        sum_fp += fp
        sum_fn += fn

        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * precision * recall / (precision + recall)

        sum_precision = sum_tp / (sum_tp + sum_fp)
        sum_recall = sum_tp / (sum_tp + sum_fn)
        sum_f1 = 2 * sum_precision * sum_recall / (sum_precision + sum_recall)

        print("CURRENT TP:", tp, " TN:", tn, " FP:", fp, " FN:", fn, " Precision:", precision, " Recall:", recall, " F1-score:", f1)
        print("OVERALL TP:", sum_tp, " TN:", sum_tn, " FP:", sum_fp, " FN:", sum_fn, 
                      " Precision:", sum_precision, " Recall:", sum_recall, " F1-score:", sum_f1)

        print("Batch Test Accuracy:" + "{:.4f}".format(acc) + " Mean Batch Test Acc:" + "{:.4f}".format(sum_acc/i))

Training...


  0%|                                                                                            | 0/6 [00:00<?, ?it/s]C:\Users\mariya.bakhanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in float_scalars
C:\Users\mariya.bakhanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in double_scalars


CURRENT TP: 0.0  TN: 162.0  FP: 0.0  FN: 152.0  Precision: nan  Recall: 0.0  F1-score: nan
OVERALL TP: 0.0  TN: 162.0  FP: 0.0  FN: 152.0  Precision: nan  Recall: 0.0  F1-score: nan
Epoch:1 Step:0 Loss:0.7029 Batch Acc:0.5159 Mean Batch Acc:0.5159



 17%|██████████████                                                                      | 1/6 [00:38<03:14, 38.88s/it]

CURRENT TP: 139.0  TN: 0.0  FP: 175.0  FN: 0.0  Precision: 0.44267517  Recall: 1.0  F1-score: 0.6136865730691926
OVERALL TP: 139.0  TN: 162.0  FP: 175.0  FN: 152.0  Precision: 0.4426751592356688  Recall: 0.47766323024054985  F1-score: 0.4595041322314049
Epoch:1 Step:1 Loss:0.7086 Batch Acc:0.4427 Mean Batch Acc:0.4793


...

 33%|████████████████████████████                                                        | 1/3 [00:08<00:16,  8.49s/it]

CURRENT TP: 146.0  TN: 129.0  FP: 27.0  FN: 12.0  Precision: 0.84393066  Recall: 0.9240506  F1-score: 0.8821752389845824
OVERALL TP: 289.0  TN: 254.0  FP: 58.0  FN: 27.0  Precision: 0.8328530259365994  Recall: 0.9145569620253164  F1-score: 0.8717948717948717
Batch Test Accuracy:0.8758 Mean Batch Test Acc:0.8646


 67%|████████████████████████████████████████████████████████                            | 2/3 [00:16<00:08,  8.46s/it]

CURRENT TP: 148.0  TN: 99.0  FP: 43.0  FN: 24.0  Precision: 0.7748691  Recall: 0.8604651  F1-score: 0.815426950243234
OVERALL TP: 437.0  TN: 353.0  FP: 101.0  FN: 51.0  Precision: 0.8122676579925651  Recall: 0.8954918032786885  F1-score: 0.8518518518518519
Batch Test Accuracy:0.7866 Mean Batch Test Acc:0.8386


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:25<00:00,  8.40s/it]
